In [91]:
#Read Input & Assign Variables
import time
start_time = time.time()
import csv
import math
with open('toy_testcase.csv', newline='') as input:
    data = tuple(csv.reader(input))

M = int(data[0][0])
N = int(data[0][1])
K = int(data[1][0])
alpha = int(data[2][0])

init_speed = ()
data_size = ()
factor = ()

for i in range (3,3+K):
    init_speed = (*init_speed, int(data[i][1]))
    data_size = (*data_size, int(data[i][2]))
    factor = (*factor, int(data[i][3]))
total_data = sum(list(data_size))
data_remaining = list(data_size)


#Global Variables for Output
grid = [[0]*M]*N
avg_speed = [[0]*2]*K
data_loss = [0]*K
penalty_term = 0
obj_function = 0
score = 0
execution_time = 0


In [92]:
with open('speed_to_data_map.csv', newline='') as mapping:
    speed_map = tuple(csv.reader(mapping))
    
speed_data_map = []
for i in range (0,len(speed_map)):
    speed_data_map = (*speed_data_map, int(speed_map[i][1]))


In [93]:
def size_index(data):
    index_map = []
    for i in range (0,K):
        max_index = data.index(max(data))
        index_map.append(max_index)
        data[max_index] = -1
    return index_map

In [94]:
def target_factor(index_map,users,flag):
    sent_data = 0
    collocated_factor_list = [0]*M
    for i in range (0,users):
        collocated_factor_list[i] = factor[index_map[i]-1]/100
    
    for i in range (0,users):
        collocated_factor = collocated_factor_list[i] * sum((collocated_factor_list[:i]+collocated_factor_list[i+1:]))
        speed = init_speed[index_map[i]-1] * (1-collocated_factor)
        speed_floor = math.floor(speed + 10e-6)
        sent_data = sent_data + speed_data_map[speed_floor]
        if flag == 1:
            data_remaining[index_map[i]-1] = data_remaining[index_map[i]-1] - speed_data_map[speed_floor]
            if (data_remaining[index_map[i]-1] < 0):
                data_remaining[index_map[i]-1] = 0
            avg_speed[index_map[i]-1][0] = ((avg_speed[index_map[i]-1][0])*avg_speed[index_map[i]-1][1] + speed)/((avg_speed[index_map[i]-1][1])+1)
            avg_speed[index_map[i]-1][1] = avg_speed[index_map[i]-1][1] + 1
            print("avg_speed: ", avg_speed)

    target_curr = (math.floor((sent_data/total_data)*100))
    return target_curr


In [95]:
def write():
    print(grid)
    print(avg_speed)
    print(data_loss)

In [96]:
target_ideal = 100/N
for i in range (0,N):       #Columns 
    user_index_map = [0]*M
    index_map_temp = [0]*M
    data_temp = data_remaining.copy()
    size_index_map = size_index(data_temp)         #shows the index of largest and the subsequent largest data_size 
    margin = 100
    for j in range (0,M):
        index_map_temp[j] = size_index_map[j]+1  
        target_curr = target_factor(index_map_temp,j+1,0)
        if abs(target_curr-target_ideal) > margin:
            target_factor(user_index_map,j,1)
            grid[i] = user_index_map.copy()
            break
        else:
            margin = abs(target_curr - target_ideal)
            user_index_map = index_map_temp.copy()
    
execution_time = time.time() - start_time
data_loss = data_remaining.copy()
write()
        

avg_speed:  [[15.2, 1], [15.2, 1], [15.2, 1], [15.2, 1]]
avg_speed:  [[17.48, 2], [17.48, 2], [17.48, 2], [17.48, 2]]
avg_speed:  [[17.653333333333332, 3], [17.653333333333332, 3], [17.653333333333332, 3], [17.653333333333332, 3]]
avg_speed:  [[16.615, 4], [16.615, 4], [16.615, 4], [16.615, 4]]
avg_speed:  [[16.892, 5], [16.892, 5], [16.892, 5], [16.892, 5]]
avg_speed:  [[16.326666666666664, 6], [16.326666666666664, 6], [16.326666666666664, 6], [16.326666666666664, 6]]
avg_speed:  [[16.508571428571425, 7], [16.508571428571425, 7], [16.508571428571425, 7], [16.508571428571425, 7]]
avg_speed:  [[16.644999999999996, 8], [16.644999999999996, 8], [16.644999999999996, 8], [16.644999999999996, 8]]
avg_speed:  [[17.251111111111108, 9], [17.251111111111108, 9], [17.251111111111108, 9], [17.251111111111108, 9]]
avg_speed:  [[16.800999999999995, 10], [16.800999999999995, 10], [16.800999999999995, 10], [16.800999999999995, 10]]
[[4, 3, 0], [4, 2, 0], [4, 2, 0], [4, 1, 0], [3, 2, 0]]
[[16.800999999